In [ ]:
%pwd

In [5]:
import os
os.chdir("../")
%pwd

'c:\\Users\\herma\\OneDrive\\Documents\\End-to-end-ML-with-mlflow'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/hsiebenou/End-to-end-ML-with-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="hsiebenou"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3dc1a66a800fb4c1281035c85ddf9375e8282b39"

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_param: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str 

In [10]:
from mlProject import logger
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
            ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            all_param=params,
            metric_file_name=config.metric_file_name,
            mlflow_uri="https://dagshub.com/hsiebenou/End-to-end-ML-with-mlflow.mlflow",
            model_path=config.model_path,
            target_column=schema.name
        )
        return model_evaluation_config


In [14]:
!pip install mlflow

In [26]:
# %pip install mlflow
# %pip install mlflow[pipelines]
# %pip install mlflow[extras]
# %pip install mlflow-skinny
# %pip install protobuf==3.20.1
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
# import mlflow.sklearn
import numpy as np 
import joblib

In [27]:
import mlflow
from mpld3 import save_json
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_url()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(Path=Path(self.config.metric_file_name), data = scores)

            mlflow.log_params(self.config.all_param)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are  other ways to use the model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/modem-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")





AttributeError: 'NoneType' object has no attribute 'extensions_by_name'

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e